Environment Setup

In [ ]:
!pip install --upgrade tensorflow_federated

Data Perturbation

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import cv2
import matplotlib.pyplot as plt

# Set random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transform for the DataLoader used for visualization (with normalization)
transform_visualize = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Download and load the CIFAR-10 dataset
dataset_path = './CIFAR10_data'
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# You can adjust the batch size and other parameters as needed
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(dataset_path, train=True, download=True, transform=transform_visualize),
    batch_size=batch_size, shuffle=True)

# Define the AlexNet architecture for CIFAR-10
class AlexNetCIFAR10(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetCIFAR10, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

# Define the learning rate
learning_rate = 0.001

# Initialize the global model
global_model = AlexNetCIFAR10().to(device)

# Define the optimizer and loss function for the global model
optimizer = optim.Adam(global_model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

# Create a directory to save the perturbed images
perturbed_dir = "perturbed_images_cifar10"
os.makedirs(perturbed_dir, exist_ok=True)

# Define the epsilon value for the DeepFool attack
epsilon = 0.03

# Modified DeepFool attack to increase perturbation
def deepfool_attack_batch(images, model, epsilon, reduction_factor=0.95):
    model.eval()
    perturbed_images = images.clone().detach().requires_grad_(True).to(device)

    with torch.no_grad():
        outputs = model(images)
        original_labels = torch.argmax(outputs, dim=1)

    outputs_grad = None
    while True:
        outputs = model(perturbed_images)
        predicted_labels = torch.argmax(outputs, dim=1)

        if torch.all(predicted_labels == original_labels):
            break

        if perturbed_images.grad is not None:
            perturbed_images.grad.data.zero_()

        loss = criterion(outputs, original_labels)
        loss.backward(retain_graph=True)

        if perturbed_images.grad is not None and outputs_grad is None:
            outputs_grad = perturbed_images.grad.data.clone()

        if outputs_grad is not None:
            perturbation = torch.abs(loss) / torch.norm(outputs_grad.view(images.size(0), -1), dim=1).view(-1, 1, 1, 1)
            perturbed_images = perturbed_images + perturbation * outputs_grad / torch.norm(outputs_grad.view(images.size(0), -1), dim=1).view(-1, 1, 1, 1)
            perturbed_images = torch.clamp(perturbed_images, 0, 1)
            perturbed_images = perturbed_images.clone().detach().requires_grad_(True).to(device)
        else:
            raise ValueError("No gradient computed for perturbed_images. The attack can't proceed.")

    return perturbed_images.detach()

# Apply the modified DeepFool attack to the entire dataset and save the perturbed images
total_images = len(train_loader.dataset)
num_batches = (total_images + batch_size - 1) // batch_size

for batch_idx, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # DeepFool attack for the batch of images
    perturbed_images = deepfool_attack_batch(images, global_model, epsilon)

    # Calculate the number of images in this batch
    num_images_in_batch = len(images)

    for j in range(num_images_in_batch):
        # Convert the perturbed image to a numpy array
        perturbed_image_np = perturbed_images[j].detach().cpu().numpy().transpose((1, 2, 0))

        # Get the class label of the image
        class_label = labels[j].item()

        # Create a subdirectory for the class label if it does not exist
        class_subdir = os.path.join(perturbed_dir, str(class_label))
        os.makedirs(class_subdir, exist_ok=True)

        # Save the perturbed image
        perturbed_image_path = os.path.join(class_subdir, f"perturbed_{batch_idx * num_images_in_batch + j}.jpg")
        perturbed_image_np = (perturbed_image_np * 255).astype(np.uint8)
        cv2.imwrite(perturbed_image_path, cv2.cvtColor(perturbed_image_np, cv2.COLOR_RGB2BGR))


Perturbed Dataset in Federated Learning Environment (using GoogleNet model)

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data.dataset import Subset
from torchvision.models import googlenet, GoogLeNet_Weights

# Set random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Check if multiple GPUs are available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transform to apply to the images
perturbed_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Specify the path to the dataset directory
perturbed_dataset_path = "./perturbed_images_cifar10/"

# Ensure the directory exists
if not os.path.exists(perturbed_dataset_path):
    raise FileNotFoundError(f"The directory {perturbed_dataset_path} does not exist.")

# Define the network architecture (GoogLeNet)
class GoogLeNet(nn.Module):
    def __init__(self, num_classes=105):
        super(GoogLeNet, self).__init__()
        # Load the pretrained GoogLeNet model
        self.model = googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)

        # Modify the final fully connected layer to match the number of classes
        self.model.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        return self.model(x)

# Wrap the model with DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for Data Parallelism.")
    global_model = nn.DataParallel(GoogLeNet()).to(device)
else:
    global_model = GoogLeNet().to(device)

# Define the federated learning parameters
num_clients = 4
fraction = 0.2
num_epochs = 200
learning_rate = 0.001

# Define the optimizer and learning rate scheduler for the global model
optimizer = optim.Adam(global_model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

criterion = nn.CrossEntropyLoss()

# Divide the perturbed dataset into train and test sets
perturbed_full_dataset = ImageFolder(perturbed_dataset_path, transform=perturbed_transform)
perturbed_num_samples = len(perturbed_full_dataset)
perturbed_num_train_samples = int(perturbed_num_samples * (1 - fraction))
perturbed_num_test_samples = perturbed_num_samples - perturbed_num_train_samples
perturbed_train_dataset, perturbed_test_dataset = random_split(perturbed_full_dataset, [perturbed_num_train_samples, perturbed_num_test_samples])

# Split the train dataset into client datasets
perturbed_train_indices = list(range(len(perturbed_train_dataset)))
perturbed_client_datasets = []
start = 0
for _ in range(num_clients):
    end = start + int(len(perturbed_train_indices) / num_clients)
    indices = perturbed_train_indices[start:end]
    subset = Subset(perturbed_train_dataset, indices)
    perturbed_client_datasets.append(subset)
    start = end

# Function to train a local model on a client's dataset
def train_local_model(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total * 100
    return model.state_dict(), train_loss, train_accuracy

# Function to aggregate model updates using federated averaging
def aggregate_models(global_model, local_models):
    global_dict = global_model.state_dict()
    for key in global_dict.keys():
        # Convert model weights to float
        global_dict[key] = torch.stack([local_models[i][key].float() for i in range(len(local_models))], dim=0).mean(0)
    global_model.load_state_dict(global_dict)
    return global_model

# Lists to store the values for plotting
train_losses_plot = []
train_accuracies_plot = []
test_losses_plot = []
test_accuracies_plot = []

# Train the global model using federated learning
for epoch in range(num_epochs):
    local_models = []
    train_losses = []
    train_accuracies = []

    # Train local models on client datasets
    for client_dataset in perturbed_client_datasets:
        train_loader = DataLoader(client_dataset, batch_size=32, shuffle=True)
        local_model = GoogLeNet().to(device)
        # Wrap the local model with DataParallel if multiple GPUs are available
        if torch.cuda.device_count() > 1:
            local_model = nn.DataParallel(local_model)

        local_model.load_state_dict(global_model.state_dict())
        local_optimizer = optim.Adam(local_model.parameters(), lr=learning_rate)

        # Learning rate scheduler for local models
        local_scheduler = optim.lr_scheduler.StepLR(local_optimizer, step_size=30, gamma=0.1)

        local_model_dict, train_loss, train_accuracy = train_local_model(local_model, train_loader, local_optimizer, criterion, device)
        local_models.append(local_model_dict)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        local_scheduler.step()

    # Aggregate the local models using federated averaging
    global_model = aggregate_models(global_model, local_models)

    # Evaluation on the test set
    test_loader = DataLoader(perturbed_test_dataset, batch_size=32, shuffle=False)
    global_model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = global_model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            test_loss += loss.item()

    test_loss /= len(test_loader)
    test_accuracy = test_correct / test_total * 100

    # Append values to the plotting lists
    train_losses_plot.append(sum(train_losses) / len(train_losses))
    train_accuracies_plot.append(sum(train_accuracies) / len(train_accuracies))
    test_losses_plot.append(test_loss)
    test_accuracies_plot.append(test_accuracy)

    # Print the epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {sum(train_losses) / len(train_losses):.4f} | Train Accuracy: {sum(train_accuracies) / len(train_accuracies):.2f}%")
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%")
    print()

# Plotting train and test loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses_plot, label='Train Loss')
plt.plot(test_losses_plot, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Test Loss')
plt.legend()
plt.grid()
plt.show()

# Plotting train and test accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_accuracies_plot, label='Train Accuracy')
plt.plot(test_accuracies_plot, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Train and Test Accuracy')
plt.legend()
plt.grid()
plt.show()
